# Fine-Tuning Gemma-3-4B on MathX-5M Dataset

This notebook demonstrates fine-tuning Google's Gemma-3-4B-IT model for mathematical reasoning using the XenArcAI/MathX-5M dataset with QLoRA (Quantized Low-Rank Adaptation).

## Overview
- **Model**: google/gemma-3-4b-it (4B parameter instruction-tuned model)
- **Dataset**: XenArcAI/MathX-5M (~4.32M mathematical problems with solutions)
- **Technique**: QLoRA with 4-bit quantization
- **Goal**: Train a math reasoning agent capable of step-by-step problem solving

## 1. Install Required Libraries

In [1]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes torch

## 2. Import Dependencies

In [2]:
import os
import torch
import gc
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.9.0+cpu
CUDA available: False


## 3. Configuration Parameters

In [ ]:
# Model and dataset configuration
MODEL_NAME = "google/gemma-3-4b-it"
DATASET_NAME = "XenArcAI/MathX-5M"

# Directory to store downloaded models and tokenizers
MODEL_DIR = "./models"

# Use a small fraction for demo/testing (remove [:1%] to use full dataset)
DATASET_SPLIT = "train[:1%]"  # ~43K examples

# Output directories
OUTPUT_DIR = "./gemma3-4b-math-lora-adapter"
MERGED_MODEL_DIR = "./gemma3-4b-math-merged"

# Training hyperparameters
MAX_SEQ_LENGTH = 2048
PER_DEVICE_BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = 8  # Effective batch size = 2 * 8 = 16
LEARNING_RATE = 2e-4
NUM_TRAIN_EPOCHS = 1
LOGGING_STEPS = 10
SAVE_STEPS = 100

# Set your Hugging Face token if needed
HF_TOKEN = os.environ.get("HF_TOKEN", None)

print(f"Model: {MODEL_NAME}")
print(f"Dataset: {DATASET_NAME} ({DATASET_SPLIT})")
print(f"Output directory: {OUTPUT_DIR}")

Model: google/gemma-3-4b-it
Dataset: XenArcAI/MathX-5M (train[:1%])
Output directory: ./gemma3-4b-math-lora-adapter


## 4. Load and Explore Dataset

The MathX-5M dataset contains:
- **problem**: Mathematical problem statement (often with LaTeX)
- **expected_answer**: Final correct answer
- **generated_solution**: Step-by-step reasoning with `<think>` tags

In [5]:
from datasets import load_dataset, Dataset
from itertools import islice

# Step 1: Load the dataset in streaming mode
streamed_dataset = load_dataset("XenArcAI/MathX-5M", split="train", streaming=True)

# Step 2: Normalize columns on the fly
def unify_columns(ex):
    if "question" in ex:
        ex["problem"] = ex.pop("question")
    return ex

streamed_dataset = streamed_dataset.map(unify_columns)

# Step 3: Take first 1% (~10k examples) and materialize in memory
subset = list(islice(streamed_dataset, 10000))

dataset = Dataset.from_list(subset).select_columns([
    "problem", "generated_solution", "expected_answer"
])

print(f"\nDataset loaded successfully!")
print(f"Number of examples: {len(dataset)}")
print(f"\nDataset features: {dataset.features}")
print(f"\nColumn names: {dataset.column_names}")

Resolving data files:   0%|          | 0/213 [00:00<?, ?it/s]


Dataset loaded successfully!
Number of examples: 10000

Dataset features: {'problem': Value('string'), 'generated_solution': Value('string'), 'expected_answer': Value('string')}

Column names: ['problem', 'generated_solution', 'expected_answer']


In [6]:
# Inspect a sample example
sample = dataset[0]

print("=" * 80)
print("SAMPLE EXAMPLE FROM DATASET")
print("=" * 80)
print(f"\nPROBLEM:\n{sample['problem'][:500]}...")
print(f"\n\nEXPECTED ANSWER:\n{sample['expected_answer']}")
print(f"\n\nGENERATED SOLUTION (first 800 chars):\n{sample['generated_solution'][:800]}...")
print("=" * 80)

SAMPLE EXAMPLE FROM DATASET

PROBLEM:
Given a group of \( N \) balls consisting of \( C \) colors, where the number of balls in each color is represented as \( n_1, n_2, \ldots, n_C \) (with \( n_1 + n_2 + \ldots + n_C = N \)), what is the probability that when \( A \) balls are randomly picked (where \( A \leq N \)), the picked balls consist of \( a_1, a_2, \ldots, a_C \) balls of each color, where \( a_1 + a_2 + \ldots + a_C = A \)?...


EXPECTED ANSWER:
\(\frac{C_{n_1}^{a_1} \cdot C_{n_2}^{a_2} \cdot \ldots \cdot C_{n_C}^{a_C}}{C_N^A}\)


GENERATED SOLUTION (first 800 chars):
<think>
Okay, so I need to find the probability that when I pick A balls out of N, where there are C different colors, the number of each color I pick is exactly a1, a2, ..., aC. Hmm, let's think about how to approach this.

First, probability problems often involve combinations. The general formula for probability is the number of favorable outcomes divided by the total number of possible outcomes. So, in this

## 5. Load Tokenizer

Load the Gemma-3 tokenizer and configure it for training.

In [ ]:
print(f"Loading tokenizer: {MODEL_NAME}...")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    token=HF_TOKEN,
    trust_remote_code=True,
    cache_dir=MODEL_DIR,
)

# Set padding token (Gemma models typically use eos_token as pad_token)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"Tokenizer loaded successfully!")
print(f"Vocab size: {len(tokenizer)}")
print(f"PAD token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print(f"BOS token: {tokenizer.bos_token} (ID: {tokenizer.bos_token_id})")

Loading tokenizer: google/gemma-3-4b-it...


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

d:\DSAI-Project\CourseGPT-Pro-DSAI-Lab-Group-6\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shubh\.cache\huggingface\hub\models--google--gemma-3-4b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Tokenizer loaded successfully!
Vocab size: 262145
PAD token: <pad> (ID: 0)
EOS token: <eos> (ID: 1)
BOS token: <bos> (ID: 2)


## 6. Format Dataset for Instruction Tuning

Format the dataset using Gemma's chat template. We'll combine the problem and solution into a conversational format.

In [10]:
def format_math_example(example):
    """
    Format a math example for instruction tuning.
    Creates a conversation with:
    - System: Math assistant role
    - User: Problem statement
    - Assistant: Step-by-step solution with final answer
    """
    # Construct the prompt
    messages = [
        {
            "role": "system",
            "content": "You are an expert mathematics tutor. Solve problems step-by-step, showing your reasoning clearly."
        },
        {
            "role": "user",
            "content": example['problem']
        },
        {
            "role": "assistant",
            "content": example['generated_solution']
        }
    ]
    
    # Apply chat template
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    return {"text": formatted_text}

print("Formatting dataset...")
formatted_dataset = dataset.map(
    format_math_example,
    remove_columns=dataset.column_names,
)

print(f"\nDataset formatted successfully!")
print(f"Number of examples: {len(formatted_dataset)}")

Formatting dataset...


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]


Dataset formatted successfully!
Number of examples: 10000


In [11]:
# Inspect a formatted example
print("=" * 80)
print("FORMATTED EXAMPLE")
print("=" * 80)
print(formatted_dataset[0]['text'][:1000])
print("...")
print("=" * 80)

FORMATTED EXAMPLE
<bos><start_of_turn>user
You are an expert mathematics tutor. Solve problems step-by-step, showing your reasoning clearly.

Given a group of \( N \) balls consisting of \( C \) colors, where the number of balls in each color is represented as \( n_1, n_2, \ldots, n_C \) (with \( n_1 + n_2 + \ldots + n_C = N \)), what is the probability that when \( A \) balls are randomly picked (where \( A \leq N \)), the picked balls consist of \( a_1, a_2, \ldots, a_C \) balls of each color, where \( a_1 + a_2 + \ldots + a_C = A \)?<end_of_turn>
<start_of_turn>model
<think>
Okay, so I need to find the probability that when I pick A balls out of N, where there are C different colors, the number of each color I pick is exactly a1, a2, ..., aC. Hmm, let's think about how to approach this.

First, probability problems often involve combinations. The general formula for probability is the number of favorable outcomes divided by the total number of possible outcomes. So, in this case, th

## 7. Configure 4-bit Quantization (QLoRA)

Set up BitsAndBytes configuration for 4-bit quantization to reduce memory usage.

In [12]:
# Determine compute dtype based on GPU capability
COMPUTE_DTYPE = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16

print(f"Compute dtype: {COMPUTE_DTYPE}")

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",               # NormalFloat 4-bit quantization
    bnb_4bit_compute_dtype=COMPUTE_DTYPE,    # Compute in BF16/FP16
    bnb_4bit_use_double_quant=True,          # Nested quantization for extra memory savings
)

print("\n4-bit quantization config:")
print(f"  - Quantization type: nf4")
print(f"  - Compute dtype: {COMPUTE_DTYPE}")
print(f"  - Double quantization: True")

Compute dtype: torch.float16

4-bit quantization config:
  - Quantization type: nf4
  - Compute dtype: torch.float16
  - Double quantization: True


## 8. Load Gemma-3 Model with Quantization

In [ ]:
print(f"Loading model: {MODEL_NAME} with 4-bit quantization...")
print("This may take a few minutes...\n")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    token=HF_TOKEN,
    torch_dtype=COMPUTE_DTYPE,
    cache_dir=MODEL_DIR,
)

# Disable caching for training
model.config.use_cache = False

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

print(f"\nModel loaded successfully!")
print(f"Model dtype: {model.dtype}")
print(f"Device map: {model.hf_device_map}")

Loading model: google/gemma-3-4b-it with 4-bit quantization...
This may take a few minutes...



config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]


Model loaded successfully!
Model dtype: torch.float32
Device map: {'': 'cpu'}


## 9. Configure LoRA

Set up Low-Rank Adaptation (LoRA) for efficient fine-tuning.

In [ ]:
lora_config = LoraConfig(
    r=16,                                    # LoRA attention dimension (Rank)
    lora_alpha=32,                           # Scaling factor (usually 2*r is a good start)
    target_modules=[                         # Target all linear layers for Gemma
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

print("LoRA Configuration:")
print(f"  - Rank (r): {lora_config.r}")
print(f"  - Alpha: {lora_config.lora_alpha}")
print(f"  - Dropout: {lora_config.lora_dropout}")
print(f"  - Target modules: {lora_config.target_modules}")
print(f"  - Task type: {lora_config.task_type}")

## 10. Configure Training Arguments

In [ ]:
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    
    # Optimizer settings
    optim="paged_adamw_8bit",                # Memory-efficient optimizer
    
    # Logging and saving
    logging_steps=LOGGING_STEPS,
    save_steps=SAVE_STEPS,
    save_strategy="steps",
    
    # Mixed precision training
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    
    # Memory optimization
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant': False},
    
    # Dataset settings
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_text_field="text",
    packing=False,  # Set to True for better efficiency if sequences are short
    
    # Other settings
    report_to="none",  # Change to "wandb" if you want to use Weights & Biases
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
)

print("Training Configuration:")
print(f"  - Epochs: {NUM_TRAIN_EPOCHS}")
print(f"  - Batch size per device: {PER_DEVICE_BATCH_SIZE}")
print(f"  - Gradient accumulation: {GRADIENT_ACCUMULATION_STEPS}")
print(f"  - Effective batch size: {PER_DEVICE_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"  - Learning rate: {LEARNING_RATE}")
print(f"  - Max sequence length: {MAX_SEQ_LENGTH}")
print(f"  - Mixed precision: {'BF16' if torch.cuda.is_bf16_supported() else 'FP16'}")

## 11. Initialize SFT Trainer

In [ ]:
print("Initializing SFT Trainer...\n")

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=formatted_dataset,
    peft_config=lora_config,
    tokenizer=tokenizer,
)

print("Trainer initialized successfully!")
print(f"\nTrainable parameters:")
trainer.model.print_trainable_parameters()

## 12. Start Training

⚠️ **Note**: Training will take time depending on your hardware and dataset size. Monitor GPU memory usage.

In [ ]:
print("=" * 80)
print("STARTING TRAINING")
print("=" * 80)
print("\nThis may take several hours depending on your hardware and dataset size.")
print("Monitor GPU usage and adjust batch size if needed.\n")

# Start training
trainer.train()

print("\n" + "=" * 80)
print("TRAINING COMPLETED!")
print("=" * 80)

## 13. Save LoRA Adapters

In [ ]:
ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter")

print(f"Saving LoRA adapters to: {ADAPTER_PATH}")

trainer.model.save_pretrained(ADAPTER_PATH)
tokenizer.save_pretrained(ADAPTER_PATH)

print(f"\nLoRA adapters saved successfully!")
print(f"Location: {ADAPTER_PATH}")

## 14. Clean Up Memory

In [ ]:
# Clear memory before loading merged model
del trainer
del model
gc.collect()
torch.cuda.empty_cache()

print("Memory cleared.")

## 15. Merge LoRA Adapters with Base Model (Optional)

Merge the LoRA adapters back into the base model for easier deployment.

In [ ]:
print("Loading base model for merging...")

# Load base model without quantization for merging
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto",
    trust_remote_code=True,
    token=HF_TOKEN,
    cache_dir=MODEL_DIR,
)

print(f"Loading LoRA adapters from: {ADAPTER_PATH}")
model_with_adapter = PeftModel.from_pretrained(base_model, ADAPTER_PATH)

print("Merging adapters with base model...")
merged_model = model_with_adapter.merge_and_unload()

print(f"\nSaving merged model to: {MERGED_MODEL_DIR}")
merged_model.save_pretrained(MERGED_MODEL_DIR)
tokenizer.save_pretrained(MERGED_MODEL_DIR)

print(f"\nMerged model saved successfully!")
print(f"Location: {MERGED_MODEL_DIR}")

## 16. Test the Fine-Tuned Model

Let's test the model with a sample math problem.

In [ ]:
# Test prompt
test_problem = """Solve the following problem step by step:

A rectangle has a length that is 3 units longer than twice its width. 
If the perimeter of the rectangle is 54 units, find the dimensions of the rectangle."""

messages = [
    {
        "role": "system",
        "content": "You are an expert mathematics tutor. Solve problems step-by-step, showing your reasoning clearly."
    },
    {
        "role": "user",
        "content": test_problem
    }
]

# Format prompt
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(merged_model.device)

print("=" * 80)
print("TESTING FINE-TUNED MODEL")
print("=" * 80)
print(f"\nProblem:\n{test_problem}\n")
print("Generating solution...\n")

# Generate response
outputs = merged_model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

# Decode response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract just the assistant's response
response_start = response.find("assistant") + len("assistant")
assistant_response = response[response_start:].strip()

print("Model Response:")
print("-" * 80)
print(assistant_response)
print("=" * 80)

## 17. Summary

### Training Complete! 🎉

**What we did:**
1. Loaded XenArcAI/MathX-5M dataset (4.32M math problems)
2. Formatted data for instruction tuning with Gemma chat template
3. Configured 4-bit quantization (QLoRA) for memory efficiency
4. Applied LoRA adapters (r=16, alpha=32) to Gemma-3-4B
5. Fine-tuned on mathematical reasoning tasks
6. Saved adapters and merged model

**Output files:**
- LoRA adapters: `./gemma3-4b-math-lora-adapter/final_adapter/`
- Merged model: `./gemma3-4b-math-merged/`

**Next steps:**
- Evaluate model on validation set
- Test on various math problem types
- Deploy as a math reasoning agent
- Optionally: Train for more epochs or on larger dataset fraction

## Appendix: Model Loading for Inference

To use the fine-tuned model later:

In [ ]:
# Example: Load merged model for inference
# from transformers import AutoModelForCausalLM, AutoTokenizer
#
# model = AutoModelForCausalLM.from_pretrained(
#     "./gemma3-4b-math-merged",
#     device_map="auto",
#     torch_dtype=torch.bfloat16
# )
# tokenizer = AutoTokenizer.from_pretrained("./gemma3-4b-math-merged")

# Or load with adapters:
# from peft import PeftModel
#
# base_model = AutoModelForCausalLM.from_pretrained(
#     "google/gemma-3-4b-it",
#     device_map="auto"
# )
# model = PeftModel.from_pretrained(
#     base_model,
#     "./gemma3-4b-math-lora-adapter/final_adapter"
# )